# **Git-Hub URL**

In [ ]:
# https://github.com/aamemara/cnn-cancer-detection.git

#**EDA**

In [14]:
from google.colab import drive
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from sklearn.model_selection import train_test_split
import os
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
# Mount drive to load files
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
drive_path = "drive/MyDrive/Colab Notebooks/University of Colorado Boulder:  Introduction to Deep Learning/"
print(os.listdir(drive_path+"histopathologic-cancer-detection"))
df_y_train_val = pd.read_csv(drive_path+"histopathologic-cancer-detection/train_labels.csv")
df_y_test = pd.read_csv(drive_path+"histopathologic-cancer-detection/sample_submission.csv")
print(df_y_train_val.info(),df_y_test.info())
print(df_y_train_val['label'].unique())
y_train_val = df_y_train_val['label'].tolist()
y_test = df_y_test['label'].tolist()

['sample_submission.csv', 'train_labels.csv', 'train', 'test']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220025 entries, 0 to 220024
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      220025 non-null  object
 1   label   220025 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57458 entries, 0 to 57457
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      57458 non-null  object
 1   label   57458 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 897.9+ KB
None None
[0 1]


In [4]:
# Create directories for image generator and save them to avoid creation in subsequent runs
train_path = drive_path+"histopathologic-cancer-detection/train"
test_path = drive_path+"histopathologic-cancer-detection/test"
# Get the list of image filenames
train_val_filenames_file =  drive_path+"histopathologic-cancer-detection/train/train_val_filenames.txt"
test_filenames_file =  drive_path+"histopathologic-cancer-detection/test/test_filenames.txt"
if os.path.exists(train_val_filenames_file) and os.path.exists(test_filenames_file):
    # Load the filenames from files
    with open(train_val_filenames_file, 'r') as f:
        X_train_val_filenames = [line.strip() for line in f.readlines()]
    with open(test_filenames_file, 'r') as f:
        X_test_filenames = [line.strip() for line in f.readlines()]
else:
    # Write directory contents to file
    with open(train_val_filenames_file, 'w') as f:
        X_train_val_filenames = os.listdir(train_path)
        for file in X_train_val_filenames:
            f.write("%s\n" % file)
    with open(test_filenames_file, 'w') as f:
        X_test_filenames = os.listdir(test_path)
        for file in X_test_filenames:
            f.write("%s\n" % file)
train_indices_file = drive_path+"histopathologic-cancer-detection/train/train_indices.npy"
val_indices_file = drive_path+"histopathologic-cancer-detection/train/val_indices.npy"
if os.path.exists(train_indices_file) and os.path.exists(val_indices_file):
    # Load the split indices from files
    train_indices = np.load(train_indices_file)
    val_indices = np.load(val_indices_file)
else:
    # Split the data and labels into training and validation sets
    train_indices, val_indices = train_test_split(
        np.arange(len(X_train_val_filenames)), test_size=0.1, shuffle=True, random_state=42)
    # Save the split indices to files
    np.save(train_indices_file, train_indices)
    np.save(val_indices_file, val_indices)
# Extract the filenames and labels corresponding to the split indices
X_train_filenames = [X_train_val_filenames[i] for i in train_indices]
X_val_filenames = [X_train_val_filenames[i] for i in val_indices]
y_train = [y_train_val[i] for i in train_indices]
y_val = [y_train_val[i] for i in val_indices]
# Create directories for training and validation data
train_dir = drive_path+"histopathologic-cancer-detection/train/train_dir"
val_dir = drive_path+"histopathologic-cancer-detection/train/val_dir"
test_dir = drive_path+"histopathologic-cancer-detection/test/test_dir"
if not os.path.exists(train_dir):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(train_dir+"/0", exist_ok=True)
    os.makedirs(train_dir+"/1", exist_ok=True)
    # Copy training images to the training directory
    for i in range(len(X_train_filenames)):
        src = os.path.join(train_path, X_train_filenames[i])
        dst = os.path.join(train_dir, str(y_train[i]), X_train_filenames[i])
        shutil.copy(src, dst)
if not os.path.exists(val_dir):
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(val_dir+"/0", exist_ok=True)
    os.makedirs(val_dir+"/1", exist_ok=True)
    # Copy validation images to the validation directory
    for i in range(len(X_val_filenames)):
        src = os.path.join(train_path, X_val_filenames[i])
        dst = os.path.join(val_dir, str(y_val[i]), X_val_filenames[i])
        shutil.copy(src, dst)
if not os.path.exists(test_dir):
    os.makedirs(test_dir, exist_ok=True)
    # Copy test images to the test directory
    for i in range(len(X_test_filenames)):
        src = os.path.join(test_path, X_test_filenames[i])
        dst = os.path.join(test_dir, X_test_filenames[i])
        shutil.copy(src, dst)
#image_shape = mpimg.imread(os.path.join(train_dir, str(y_train[0]), X_train_filenames[0])).shape
image_shape = (96, 96, 3)

In [ ]:
# Display first 10 train images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()
for i in range(10):
    if i < 5:
      # Display 5 negative training examples
      img = mpimg.imread(train_dir+"/0/"+os.listdir(train_dir+"/0")[i])
      axes[i].set_title("0")
    else:
      # Display 5 positive training examples
      img = mpimg.imread(train_dir+"/1/"+os.listdir(train_dir+"/1")[i-5])
      axes[i].set_title("1")
    axes[i].imshow(img)
    axes[i].axis('off')
plt.tight_layout()
plt.show()

#**Model building and training**

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_shape[0], image_shape[1]),
    batch_size=32,
    class_mode='binary')
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(image_shape[0], image_shape[1]),
    batch_size=32,
    class_mode='binary')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(image_shape[0], image_shape[1]),
    batch_size=1,
    class_mode='binary')

Found 196232 images belonging to 2 classes.
Found 22003 images belonging to 2 classes.
Found 57458 images belonging to 1 classes.


###**Sequential Model**

In [5]:
model = Sequential()
model.add(Conv2D(128, (3, 3), activation = 'relu', input_shape = image_shape))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation = "sigmoid"))
model.summary()
model.compile(Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 94, 94, 128)       3584      
                                                                 
 conv2d_1 (Conv2D)           (None, 92, 92, 128)       147584    
                                                                 
 conv2d_2 (Conv2D)           (None, 90, 90, 128)       147584    
                                                                 
 max_pooling2d (MaxPooling2  (None, 45, 45, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 45, 45, 128)       0         
                                                                 
 flatten (Flatten)           (None, 259200)            0         
                                                        

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size)

###**Results**

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
predictions = model.predict(test_gen, steps=len(y_test), verbose=1)
df_y_test['label']=predictions
df_y_test.to_csv(drive_path+"histopathologic-cancer-detection/my_sample_submission.csv", index=False)

###**Pre-trained Model**

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [23]:
# Load pretrained DenseNet-201 model
base_model = DenseNet201(weights='imagenet', input_shape=image_shape, include_top=False)
# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
# Create model
model = Model(inputs=base_model.input, outputs=predictions)
# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False
model.summary()
model.compile(Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 zero_padding2d_14 (ZeroPad  (None, 102, 102, 3)          0         ['input_8[0][0]']             
 ding2D)                                                                                          
                                                                                                  
 conv1/conv (Conv2D)         (None, 48, 48, 64)           9408      ['zero_padding2d_14[0][0]']   
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 48, 48, 64)           256       ['conv1/conv[0][0]']    

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size)

###**Results**

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
predictions = model.predict(test_gen, steps=len(y_test), verbose=1)
df_y_test['label']=predictions
df_y_test.to_csv(drive_path+"histopathologic-cancer-detection/my_sample_submission.csv", index=False)

##**Conclusion**

In [ ]:
'''
In conclusion, pre-trained model has better accuracy than the first sequential model due to its large network that was trained on ImageNet and fine tuned to histopathologic cancer detection
'''